In [1]:
from film_dataset import FilmDataset

In [2]:
# df = pd.read_csv('data/movie_db.csv')
# train_df, test_df = train_test_split(pd.read_csv('data/movie_db.csv').dropna())
# train_df.to_csv('data/train_df.csv', index=False)
# test_df.to_csv('data/test_df.csv', index=False)

In [3]:
train_dataset = FilmDataset('data/train_df.csv', 10000)
train_dataset.process_plots(True, True)
train_dataset.encode_plots(True)
train_dataset.set_genres()
wrd2idx, idx2wrd, genre2idx, idx2genre = train_dataset.extract_encoders()

In [4]:
test_dataset = FilmDataset('data/test_df.csv', 5000)
test_dataset.process_plots(False, True)
test_dataset.set_encoders(wrd2idx, idx2wrd, genre2idx, idx2genre)
test_dataset.encode_plots(True)

In [5]:
import numpy as np

In [6]:
np.array(train_dataset.encoded_plots)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_dataset.processed_plots)

In [9]:
tfidf_vectorizer = TfidfVectorizer()

In [10]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm

In [11]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

NameError: name 'categories' is not defined

In [ ]:
genres = []
for i in tqdm(train_dataset):
    try:
        genres.append(np.array(i['genre']))
    except:
        continue

NameError: name 'generes' is not defined

In [ ]:
def decode_plot(idx2wrd, plot):
    
    plot = [int(i) for i in list(plot)]
    decoded = [idx2wrd[i] for i in plot if i != 0]
    
    return ' '.join(decoded[1:-1])

In [ ]:
from torch.utils.data import DataLoader
import torch

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
for i in train_dataloader:
    text = i['plot']
    image = i['poster']
    
    break

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_out) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, n_out)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        
        x = self.embeddings(x)
        x = self.dropout(x)
        
        lstm_out, (ht, ct) = self.lstm(x)
        
        return self.linear(ht[-1])

In [ ]:
vocab_size = len(wrd2idx)
hidden_dim = 512
embedding_dim = 512
n_hidden = 2
n_out = len(genre2idx)
device = 'cuda:1'

In [ ]:
import torch.nn as nn

In [ ]:
rnn_model = LSTM_fixed_len(vocab_size, embedding_dim, hidden_dim, n_out)

In [ ]:
rnn_model.to(device)
#hidden = torch.zeros(50, n_hidden).to(device)

In [ ]:
import torch.optim as optim
from tqdm.notebook import tqdm

criterion = nn.BCEWithLogitsLoss()#nn.CrossEntropyLoss()#BCEWithLogitsLoss()#
optimizer = optim.SGD(rnn_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 5

In [ ]:
# class MltcModel(torch.nn.Module):
#     def __init__(self, vocab_size, emb_dim, hid_dim, n_out, rnn_num_layers=1):
#         super().__init__()

#         self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
#         self.rnn = torch.nn.GRU(emb_dim, hid_dim, bidirectional=True, num_layers=rnn_num_layers)
#         self.l1 = torch.nn.Linear(hid_dim * 2 * rnn_num_layers, emb_dim)
#         self.l2 = torch.nn.Linear(emb_dim, n_out)

#     def forward(self, samples):
#         embedded = self.embedding(samples)

#         _, last_hidden = self.rnn(embedded)

#         hidden_list = [last_hidden[i, :, :] for i in range(last_hidden.size()[0])]
#         encoded = torch.cat(hidden_list, dim=1)

#         encoded = torch.nn.functional.relu(self.l1(encoded))
#         encoded = torch.nn.functional.sigmoid(self.l2(encoded))

#         return encoded

# rnn_model = MltcModel(vocab_size, embedding_dim, hidden_dim, n_out,rnn_num_layers=2)

# criterion = torch.nn.MultiLabelSoftMarginLoss()
# optimizer = torch.optim.Adam(rnn_model.parameters())


In [ ]:
rnn_model.to(device)
rnn_model.train()

for i in tqdm(range(0, num_epochs)):
    running_loss = []
    for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

        # get the inputs

        inputs = data['plot'].to(device)
        labels = data['genre'].to(device).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = rnn_model(inputs).float()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss.append(loss.item())
        
    print('Avg Loss: {}'.format(sum(running_loss)/len(running_loss)))



In [ ]:
rnn_model.eval()
num = 0
for i in test_dataloader:
    if num == 2:
        break
    x = i['plot'].to(device)
    out = rnn_model(x).squeeze(1)
    num+=1

In [ ]:
_, index = torch.max(x, dim=1)


In [ ]:
outputs.shape

In [ ]:
out.shape

In [ ]:
Sigmoid(out)

In [ ]:
torch.round(out[0])

In [ ]:
from torch.nn import Sigmoid